In [5]:
import pandas as pd
import numpy as np

## load data

In [62]:

def first_aid(df):
    temp = 'data/'+df+'.csv'
    df = pd.read_csv(temp , sep =',', names = list(range(0,30)))
    df = df[[4,0,5,1,2]+list(range(6,30))]
    df[5] = df[5].map(lambda x: x.lstrip('מספר תגובות -'))
    df.columns = ['url','data','number','name','title',1,2,3,1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,18,    19,20,21,22,23]
    return(df)

pain = first_aid('pain')
bowel_diseases = first_aid('bowel_diseases')
joints = first_aid('joints')
psoriasis = first_aid('psoriasis')
sclerosis = first_aid('sclerosis')

In [63]:
pain.head()

,url,data,number,name,title,1,2,3,1,2,...,14,15,16,17,18,19,20,21,22,23
0,https://www.camoni.co.il/411804/121662,02/01/2015,2,עסאף,כאב נירופטי ממקור לא ידוע,"XXלימור חורשXX\nשלום עסאף, קודם כל רק בריאות ו...","XXד""ר גור רותXX\nשלום עאטף,&nbsp;\r\n\r\nאנא פ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.camoni.co.il/411804/121678,02/01/2015,14,riki2,"כאבים ""בכל הגוף""","XXלימור חורשXX\nהיי, מטופלת אצל מרפאת כאב? תרג...",XXמינרבה1XX\nבפרטי....לא בפורום....\r\n,XXriki2XX\nתודה רבה לכל המגיבים אבל אני לא רוצ...,XXלימור חורשXX\nמרפאת כאב לא בהכרח מוסיפה תרופ...,XXרוחמהXX\nהאם מישהו ניסה את המדקות\r\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.camoni.co.il/411804/121781,10/01/2015,3,"ד""ר דינה ראלט PhD",שמיעת מוסיקה מפחיתה כאב,XXdvora30XX\nמוסיקה מנעימה את הזמן ועוזרת ברגע...,"XXד""ר דינה ראלט PhDXX\nלפי המאמר, זה יותר מרק ...","XXdvora30XX\nכן, אני יודעת מנסיון , זה עוזר לה...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.camoni.co.il/411804/121785,10/01/2015,7,אביעדאעיבד,כאב בישיבה מאזור פי הטבעת,"XXד""ר גור רותXX\nשלום רב,&nbsp;\r\n\r\nניתן לנ...",XXאביעדאעיבדXX\nעושה רושם שהגדלת מינון ליריקה ...,"XXד""ר גור רותXX\nשלום רב,&nbsp;\r\n\r\nליריקה ...",XXאביעדאעיבדXX\nתודה רבה. אני משתדל לקחת פעם א...,"XXד""ר גור רותXX\nשלום לך,\r\n\r\n&nbsp;אין שום...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.camoni.co.il/411804/121792,10/01/2015,10,נימ,כאבי תופת בשיניים (לסת חניכיים ועוד) ממקור לא ...,"XXלימור חורשXX\nהיי נימ, יש לי מישהי בקבוצה שס...","XXד""ר גור רותXX\nהיי נים,&nbsp;\r\n\r\nאם כבר ...",XXלימור חורשXX\nהאם בדקת את האבחנה&nbsp;נוירלג...,"XXנימXX\nד""ר גור שלום (שוב),\nאין המון מה לעדכ...","XXד""ר גור רותXX\nהמון בהצלחה!\r\n\r\nד&quot;ר ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
